<a href="https://colab.research.google.com/github/kurt980/OA_practice/blob/main/BCG_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from sklearn.utils import resample

In [ ]:
import io
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [ ]:
df = pd.read_csv("data.csv", encoding = 'unicode_escape')

In [ ]:
# sample 1000 data points
df1 = resample(df, n_samples=1000)

In [ ]:
# drop index
df1 = df1.reset_index(drop = True)

In [ ]:
# assign channel variable
df1 = pd.concat([df1, pd.DataFrame(np.random.randint(1,6,size=(1000, 1)), columns=['Channel'])], axis = 1)

In [ ]:
# change to datetime object
df1['InvoiceDate'] = pd.to_datetime(df1['InvoiceDate'])

In [ ]:
# create customer table
df2 = pd.DataFrame(df1['CustomerID'].dropna().unique(), columns = ['CustomerID'])

In [ ]:
# assign random date of birth
def assign_dob(key):
  year = np.random.randint(1940,2001)
  month = np.random.randint(1,12)
  day = np.random.randint(1,28)
  date = datetime.datetime(year,month,day)

  return date

df2['DateofBirth'] = df2['CustomerID'].apply(assign_dob)

In [ ]:
df2.head()

,CustomerID,DateofBirth
0,17870.0,1992-09-07
1,16240.0,1953-04-21
2,14796.0,1953-02-23
3,14504.0,1958-10-27
4,14005.0,1968-06-25


In [ ]:
df1.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Channel
0,561519,22536,MAGIC DRAWING SLATE PURDEY,1,2011-07-27 17:01:00,0.42,17870.0,United Kingdom,5
1,558240,22620,4 TRADITIONAL SPINNING TOPS,1,2011-06-27 15:27:00,2.46,NaN,United Kingdom,3
2,568065,22965,3 TRADITIONAl BISCUIT CUTTERS SET,6,2011-09-23 13:44:00,2.10,16240.0,United Kingdom,4
3,561195,23121,PACK OF 6 COCKTAIL PARASOL STRAWS,4,2011-07-25 13:57:00,0.42,14796.0,United Kingdom,3
4,547800,85152,HAND OVER THE CHOCOLATE SIGN,1,2011-03-25 12:55:00,2.10,14504.0,United Kingdom,4


Problem 1. Difference between revenues

In [ ]:
# create new column
df1['Revenue'] = df1[['Quantity', 'UnitPrice']].product(axis=1)

In [ ]:
# calculate sum
sum2011 = sum(df1[(df1['InvoiceDate'] > '12/01/2011') & (df1['InvoiceDate'] < '12/31/2011')]['Revenue'])
sum2010 = sum(df1[(df1['InvoiceDate'] > '12/01/2010') & (df1['InvoiceDate'] < '12/31/2010')]['Revenue'])
revdiff = sum2011/sum2010 - 1
print(revdiff)

-0.3350262963033286


Problem 2. Median Age per Channel

In [ ]:
df3 = pd.merge(df1, df2, on="CustomerID")

In [ ]:
df3

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Channel,Revenue,DateofBirth
0,561519,22536,MAGIC DRAWING SLATE PURDEY,1,2011-07-27 17:01:00,0.42,17870.0,United Kingdom,5,0.42,1992-09-07
1,578707,21115,ROSE CARAVAN DOORSTOP,2,2011-11-25 11:20:00,6.75,17870.0,United Kingdom,4,13.50,1992-09-07
2,572569,23510,MINI PLAYING CARDS GYMKHANA,1,2011-10-25 10:32:00,0.42,17870.0,United Kingdom,3,0.42,1992-09-07
3,566091,22983,CARD BILLBOARD FONT,24,2011-09-09 10:37:00,0.42,17870.0,United Kingdom,4,10.08,1992-09-07
4,568065,22965,3 TRADITIONAl BISCUIT CUTTERS SET,6,2011-09-23 13:44:00,2.10,16240.0,United Kingdom,4,12.60,1953-04-21
...,...,...,...,...,...,...,...,...,...,...,...
747,555853,85135C,RED DRAGONFLY HELICOPTER,1,2011-06-07 13:44:00,7.95,14375.0,United Kingdom,2,7.95,1974-08-02
748,573829,47591D,PINK FAIRY CAKE CHILDRENS APRON,8,2011-11-01 11:35:00,1.95,16830.0,United Kingdom,2,15.60,1982-02-12
749,553145,21500,PINK POLKADOT WRAP,25,2011-05-13 13:50:00,0.42,14342.0,United Kingdom,4,10.50,1993-09-24
750,563155,20675,BLUE POLKADOT BOWL,10,2011-08-12 11:48:00,1.25,13069.0,United Kingdom,1,12.50,1942-02-13


In [ ]:
# get age
now = pd.Timestamp('now')
df3['DateofBirth'] = pd.to_datetime(df3['DateofBirth'], format='%m%d%y')
df3['Age'] = (now - df3['DateofBirth']).astype('<m8[Y]')

In [ ]:
df3.groupby(['Channel'])[['Age']].median()

,Age
Channel,
1,53.0
2,54.5
3,54.0
4,53.0
5,50.0


Problem 3. Most popular product


In [ ]:
df3[(df3['DateofBirth'] > '01/01/1981') & (df3['DateofBirth'] < '12/31/1996')].groupby(['StockCode'])[['Quantity']].sum().sort_values(by = ['Quantity'], ascending = False)

,Quantity
StockCode,
84997D,360
84997A,144
84879,104
22378,50
22558,48
...,...
21823,1
21890,1
22550,-1


Problem 4. Linear Regression

Problem 6. 

Q1. aggregate revenue by year and customer id

In [ ]:
df4 = df3

In [ ]:
df4['InvoiceYear'] = df4.InvoiceDate.dt.year

In [ ]:
df4

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Channel,Revenue,DateofBirth,Age,InvoiceYear
0,561519,22536,MAGIC DRAWING SLATE PURDEY,1,2011-07-27 17:01:00,0.42,17870.0,United Kingdom,5,0.42,1992-09-07,30.0,2011-01-01
1,578707,21115,ROSE CARAVAN DOORSTOP,2,2011-11-25 11:20:00,6.75,17870.0,United Kingdom,4,13.50,1992-09-07,30.0,2011-01-01
2,572569,23510,MINI PLAYING CARDS GYMKHANA,1,2011-10-25 10:32:00,0.42,17870.0,United Kingdom,3,0.42,1992-09-07,30.0,2011-01-01
3,566091,22983,CARD BILLBOARD FONT,24,2011-09-09 10:37:00,0.42,17870.0,United Kingdom,4,10.08,1992-09-07,30.0,2011-01-01
4,568065,22965,3 TRADITIONAl BISCUIT CUTTERS SET,6,2011-09-23 13:44:00,2.10,16240.0,United Kingdom,4,12.60,1953-04-21,69.0,2011-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,555853,85135C,RED DRAGONFLY HELICOPTER,1,2011-06-07 13:44:00,7.95,14375.0,United Kingdom,2,7.95,1974-08-02,48.0,2011-01-01
748,573829,47591D,PINK FAIRY CAKE CHILDRENS APRON,8,2011-11-01 11:35:00,1.95,16830.0,United Kingdom,2,15.60,1982-02-12,40.0,2011-01-01
749,553145,21500,PINK POLKADOT WRAP,25,2011-05-13 13:50:00,0.42,14342.0,United Kingdom,4,10.50,1993-09-24,29.0,2011-01-01
750,563155,20675,BLUE POLKADOT BOWL,10,2011-08-12 11:48:00,1.25,13069.0,United Kingdom,1,12.50,1942-02-13,80.0,2011-01-01


In [ ]:
df4['InvoiceYear'] = pd.to_datetime(df4['InvoiceYear'], format='%Y')

In [ ]:
df5 = df4[['CustomerID','Revenue','InvoiceYear','Age','DateofBirth']].groupby(['CustomerID','InvoiceYear','DateofBirth']).sum().reset_index()

In [ ]:
df5

,CustomerID,InvoiceYear,DateofBirth,Revenue,Age
0,12347.0,2011-01-01,1984-04-17,17.7,38.0
1,12360.0,2011-01-01,1957-01-19,15.3,65.0
2,12378.0,2011-01-01,1963-02-25,17.7,59.0
3,12397.0,2011-01-01,1961-07-10,8.4,61.0
4,12409.0,2011-01-01,1969-02-13,183.6,53.0
...,...,...,...,...,...
580,18257.0,2011-01-01,1964-08-07,15.8,58.0
581,18263.0,2011-01-01,1965-05-13,20.0,57.0
582,18272.0,2011-01-01,1984-01-25,25.5,38.0
583,18282.0,2011-01-01,2000-10-17,17.7,21.0


Q2. Create Features

In [ ]:
df6 = df5[['Revenue','InvoiceYear','CustomerID']].groupby(['InvoiceYear','CustomerID']).sum().reset_index()

In [ ]:
df6

,InvoiceYear,CustomerID,Revenue,prev_year_revenue
0,2010-01-01,12423.0,10.08,NaN
1,2010-01-01,12433.0,13.92,10.08
2,2010-01-01,12472.0,10.20,NaN
3,2010-01-01,12662.0,17.40,NaN
4,2010-01-01,12748.0,2.95,NaN
...,...,...,...,...
580,2011-01-01,18257.0,15.80,NaN
581,2011-01-01,18263.0,20.00,NaN
582,2011-01-01,18272.0,25.50,NaN
583,2011-01-01,18282.0,17.70,NaN


In [ ]:
for i in range(np.shape(df6)[0]):
  year = df6.loc[i,'InvoiceYear'].year
  ID = df6.loc[i,'CustomerID']
  df6.loc[i,'prev_year_revenue'] = df6[(df6['InvoiceYear'] == datetime.datetime(year - 1, 1, 1)) & (df6['CustomerID'] == ID)].loc[0,'Revenue']

KeyError: ignored

In [ ]:
year = df6.loc[1,'InvoiceYear'].year
ID = df6.loc[1,'CustomerID']

In [ ]:
df6.loc[1,'prev_year_revenue'] = df6[(df6['InvoiceYear'] == datetime.datetime(2011 - 1, 1, 1)) & (df6['CustomerID'] == 12423)].loc[0,'Revenue']

In [ ]:
df6[(df6['InvoiceYear'] == datetime.datetime(2011 - 1, 1, 1))]

In [ ]:
# get revenue by year
def getYearRevenue(date):
  year = date.year
  yearlyrev = df3[['InvoiceDate','Revenue']]
  yearlyrev['InvoiceYear'] = yearlyrev.InvoiceDate.dt.year
  yearlyrev = yearlyrev[['InvoiceYear', 'Revenue']].groupby(yearlyrev['InvoiceYear']).sum()
  return 

In [ ]:
x = datetime.datetime(2011,1,1)
x.year

2011

Q3. add features

Add 